In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 16 14:51:03 2019

@author: nikhilyadav
"""

import util_mnist_reader
import numpy as np
from sklearn.metrics import confusion_matrix 
import matplotlib.pyplot as plt 
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Dense, Dropout
from keras.models import Sequential


def task1(xtrain, ytrain, xtest, ytest): #One hidden layer from scratch
    
    def sigmoid(z):
        return 1 / (1 + np.exp(-z)) 

    def loss_computation(yt, a2):
        total_loss = np.sum(np.multiply(yt, np.log(a2)))
        m = yt.shape[1]
        loss = -(1/m) * total_loss
        return loss

    
    xtrain = xtrain / 255       #normalization
    op_classes = 10
    data_size = ytrain.shape[0]
    ytrain = ytrain.reshape(1, data_size)
    y = np.eye(op_classes)[ytrain.astype('int32')] #one hot encoded
    ytrain = y.T
    ytrain = ytrain.reshape(op_classes, data_size)
    
    d = 60000
    
    xtrain = xtrain.T
    
    #hyper-parameters
    LR = 1
    hidden_nodes = 30
    epochs = 2000
    
    nodes = xtrain.shape[0]
    #setting dimensions and random values for weights and zeroes for biases
    
    
    w1 = np.random.randn(hidden_nodes, nodes)
    b1 = np.zeros((hidden_nodes, 1))
    w2 = np.random.randn(op_classes, hidden_nodes)
    b2 = np.zeros((op_classes, 1))
    
    xt = xtrain
    yt = ytrain
    losstrack = []
    
    for i in range(epochs):
                                                    #forward prop
        z1 = np.matmul(w1,xt) + b1
        a1 = sigmoid(z1)
        z2 = np.matmul(w2,a1) + b2
        a2 = np.exp(z2) / np.sum(np.exp(z2), axis=0) #softmax
    
        loss = loss_computation(yt, a2)
        losstrack.append(loss)                      #backprop
        dz2 = a2-yt
        dw2 = (1./d) * np.matmul(dz2, a1.T)
        db2 = (1./d) * np.sum(dz2, axis=1, keepdims=True)
    
        da1 = np.matmul(w2.T, dz2)
        dz1 = da1 * sigmoid(z1) * (1 - sigmoid(z1))
        dw1 = (1./d) * np.matmul(dz1, xt.T)
        db1 = (1./d) * np.sum(dz1, axis=1, keepdims=True)
    
        w2 = w2 - LR * dw2                         #update weights and biases
        b2 = b2 - LR * db2
        w1 = w1 - LR * dw1
        b1 = b1 - LR * db1
    
    print("Cost: ", loss)
    

    ##############  Predictions
    
    ytest_size = ytest.shape[0]
    xtest = xtest.T
    ytest = ytest.reshape(1, ytest_size)
    ynew_test = np.eye(op_classes)[ytest.astype('int32')]
    ytest = ynew_test.T.reshape(op_classes, ytest_size)
    
    z1 = np.matmul(w1, xtest) + b1
    a1 = sigmoid(z1)
    z2 = np.matmul(w2, a1) + b2
    a2 = np.exp(z2) / np.sum(np.exp(z2), axis=0)
    
    pred = np.argmax(a2, axis=0)
    y_op = np.argmax(ytest, axis=0)
    

    diag_sum = 0
    total_sum = 0
    cm = confusion_matrix(y_op, pred)
                                            #for accuracy calc
    print()
    for i in range (10):
        for j in range (10):
            if (i == j):
                diag_sum+=cm[i][j]
            total_sum += cm[i][j]
    
    
    
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.plot(losstrack)
    plt.show()

    print()
    print("Confusion Matrix: ")
    print()
    print(cm)
    print()
    print("ACCURACY:   ", diag_sum/total_sum)
    
    


def task2(xtrain, ytrain, xtest, ytest):            #Multilayer NN
    
    ytest_old = ytest
    
    xtrain = xtrain.reshape([-1,28,28])
    xtest = xtest.reshape([-1,28,28])
    
    
    
    xtrain = xtrain/255
    xtest = xtest/255

    
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        #keras.layers.Dense(128, activation='sigmoid'),
        keras.layers.Dense(128, activation='relu'),     #add dense layers
        keras.layers.Dense(10, activation='softmax')    #final later using softmax
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    hist = model.fit(xtrain, ytrain, epochs=100)
    
    loss1 = hist.history['loss']
    
    
    loss2, accuracy = model.evaluate(xtest,  ytest, verbose=2)  #testing loss and accuracy

    
    predict = model.predict(xtest)

    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.plot(loss1)
    plt.show()
    
    diag_sum = 0
    total_sum = 0
    
    
    cm = confusion_matrix(ytest_old, np.argmax(predict, axis = 1))
                                                                #accuracy calc
    for i in range (10):
        for j in range (10):
            if (i == j):
                diag_sum+=cm[i][j]
            total_sum += cm[i][j]
    
    print()
    print("Cost: ", loss2)
    print()
    print("Confusion Matrix: ")
    print()
    print(cm)
    print()
    print("ACCURACY:   ", diag_sum/total_sum)


def task3(xtrain, ytrain, xtest, ytest):

    ytest_old = ytest
    xtrain = xtrain.reshape([60000,28,28,1])
    xtest = xtest.reshape([10000,28,28,1])
    
    
    ytrain = keras.utils.to_categorical(ytrain)
    ytest = keras.utils.to_categorical(ytest)
    
    
    xtrain = xtrain/255
    xtest = xtest/255
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), input_shape= (28, 28, 1)))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
    model.add(Dense(60, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation='softmax'))
    
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    
    
    hist = model.fit(xtrain, ytrain, epochs=5, verbose=2)
    
    loss1, accuracy = model.evaluate(xtest,  ytest, verbose=2)
    predict = model.predict(xtest)
    
   
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.plot(hist.history['loss'])
    plt.show()
    
    diag_sum = 0
    total_sum = 0
    
    
    cm = confusion_matrix(ytest_old, np.argmax(predict, axis = 1))
                                                            #accuracy calc
    for i in range (10):
        for j in range (10):
            if (i == j):
                diag_sum+=cm[i][j]
            total_sum += cm[i][j]
    
    print()
    print("Cost: ",loss1)
    print()
    print("Confusion Matrix: ")
    print()
    print(cm)
    print()
    print("ACCURACY:   ", diag_sum/total_sum)
    



if __name__ == "__main__":
    
    
    xtrain, ytrain = util_mnist_reader.load_mnist('../data/fashion', kind='train')
    xtest, ytest = util_mnist_reader.load_mnist('../data/fashion', kind='t10k')

    task1(xtrain, ytrain, xtest, ytest)
    task2(xtrain, ytrain, xtest, ytest)
    task3(xtrain, ytrain, xtest, ytest)

Using TensorFlow backend.


NameError: name 'task1' is not defined